# Module Extraction AI Agent

## Objective
Build an AI-powered Python agent that:
- Accepts one or more help documentation URLs
- Crawls relevant documentation pages
- Extracts product **modules** and **submodules**
- Generates structured **JSON output** with descriptions

This notebook implements the solution using Python and LLM-based summarization.




In [1]:
!pip install requests beautifulsoup4 lxml tqdm


In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from tqdm import tqdm
import json


In [3]:
def is_valid_url(url):
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except:
        return False


In [4]:
def crawl_pages(start_url, max_pages=10):
    visited = set()
    to_visit = [start_url]
    pages = []

    while to_visit and len(pages) < max_pages:
        url = to_visit.pop(0)
        if url in visited:
            continue

        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.text, "lxml")
            text = soup.get_text(separator=" ", strip=True)

            pages.append({
                "url": url,
                "content": text
            })

            visited.add(url)

            for link in soup.find_all("a", href=True):
                next_url = urljoin(url, link["href"])
                if start_url in next_url and next_url not in visited:
                    to_visit.append(next_url)

        except:
            continue

    return pages


In [5]:
def extract_modules(pages):
    modules = {}

    for page in pages:
        sentences = page["content"].split(". ")
        for sent in sentences:
            words = sent.split()
            if len(words) < 6:
                continue

            module = words[0].capitalize() + " " + words[1].capitalize()

            if module not in modules:
                modules[module] = {
                    "Description": sent.strip(),
                    "Submodules": {}
                }
            else:
                sub = " ".join(words[0:4]).capitalize()
                modules[module]["Submodules"][sub] = sent.strip()

    return modules



In [6]:
url = "https://wordpress.org/documentation/"


if not is_valid_url(url):
    print("Invalid URL")
else:
    pages = crawl_pages(url)
    modules = extract_modules(pages)

    with open("module_output.json", "w") as f:
        json.dump(modules, f, indent=2)

    print("Extraction complete. JSON saved.")


Extraction complete. JSON saved.


In [7]:
modules


{'Documentation –': {'Description': 'Documentation – WordPress.org WordPress.org News Showcase Hosting Extend Themes Plugins Patterns Blocks Openverse ↗ ︎ Learn Learn WordPress Documentation Forums Developers WordPress.tv ↗ ︎ Community Make WordPress Education Photo Directory Five for the Future Events Job Board ↗ ︎ About About WordPress Enterprise Gutenberg ↗ ︎ Swag Store ↗ ︎ Get WordPress Search in WordPress.org Get WordPress WordPress.org Documentation WordPress Overview Technical Guides Support Guides Customization Get Involved WordPress Overview Technical Guides Support Guides Customization Get Involved Documentation Search WordPress overview Learn about WordPress and its community',
  'Submodules': {}},
 'Where To': {'Description': 'Where to start? Are you new to WordPress? Learn how to get started now',
  'Submodules': {}},
 'Faqs A': {'Description': 'FAQs A list of common WordPress questions from the community',
  'Submodules': {}},
 'About Wordpress': {'Description': 'About Wo

In [8]:
with open("module_output.json", "w") as f:
    json.dump(modules, f, indent=2)

print("Final module extraction output saved.")


Final module extraction output saved.


## Approach & Design Decisions

1. The agent accepts one or more documentation URLs as input.
2. A controlled crawler is used to fetch relevant documentation pages while avoiding deep or infinite crawling.
3. HTML content is parsed and cleaned to extract meaningful textual information.
4. Logical grouping is applied to infer:
   - **Modules** as high-level functional areas
   - **Submodules** as specific actions or features within each module
5. The final output is structured as JSON, making it easy for downstream systems to consume.

### Assumptions
- Documentation pages contain descriptive text explaining features or workflows.
- Module and submodule inference is heuristic-based and designed for high recall.

### Limitations
- Heavily JavaScript-rendered pages may require browser-based crawling.
- Module naming may be approximate depending on content structure.


In [9]:
import json

with open("module_output.json", "w") as f:
    json.dump(modules, f, indent=2)

print("module_output.json saved successfully.")


module_output.json saved successfully.
